In [ ]:
import os

import geoutils as gu
import numpy as np

from itertools import product
from matplotlib import pyplot as plt
from scipy.ndimage import convolve

from helpers import getV, make_affine, make_affine_matrix_1d, make_affine_matrix_2d, yield_map

# Parameters & Settings

In [ ]:
np.set_printoptions(linewidth=200)

fns = [fn for fn in os.listdir("data") if fn.endswith(".tif")]
rasters = sorted([gu.Raster(os.path.join("data", fn)) for fn in fns], key=lambda r: r.bounds[:2])

x0, z0 =  -170, -196
x1, z1 = -1144,  284

w0, h0 = 2533405,   1152605.5
w1, h1 = 2532702.5, 1152259.5

lonField, latField = 2534190, 1152375
lonEspla, latEspla = 2533030, 1152475

v0 = getV(rasters, lonField, latField)
v1 = getV(rasters, lonEspla, latEspla)

y0, y1 = 35, 67

mX, hX = make_affine(x0, w0, x1, w1)
mZ, hZ = make_affine(z0, h0, z1, h1)

m2c = make_affine_matrix_2d(mX, mZ, hX, hZ)

mX, hX = make_affine(w0, x0, w1, x1)
mZ, hZ = make_affine(h0, z0, h1, z1)

c2m = make_affine_matrix_2d(mX, mZ, hX, hZ)

mY, hY = make_affine(v0, y0, v1, y1)

v2y = make_affine_matrix_1d(mY, hY)

prefix = "summon falling_block ~ ~1 ~ {Time:1,BlockState:{Name:redstone_block},Passengers:[\
{id:armor_stand,Health:0,Passengers:[\
{id:falling_block,Time:1,BlockState:{Name:activator_rail},Passengers:[\
{id:command_block_minecart,Command:'gamerule commandBlockOutput false'},\
{id:command_block_minecart,Command:'data merge block ~ ~-2 ~ {auto:0}'},"

template = "{{id:command_block_minecart,Command:'{0}'}}"

fill_command = "fill {0} {1} {2} {3} {4} {5} {6} {7} {8}"

clone_command = "clone {0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10}"

suffix = ",{id:command_block_minecart,Command:'setblock ~ ~1 ~ command_block{auto:1,Command:\"fill ~ ~ ~ ~ ~-2 ~ air\"}'},\
{id:command_block_minecart,Command:'kill @e[type=command_block_minecart,distance=..1]'}]}]}]}"

strings = {
    "prefix": prefix,
    "template": template,
    "fill_command": fill_command,
    "clone_command": clone_command,
    "suffix": suffix
}

# Command Generation

In [ ]:
xMap, zMap = -6, 0

yield_map(rasters, strings, m2c, c2m, v2y, xMap, zMap)

In [ ]:
summon falling_block ~ ~1 ~ {Time:1,BlockState:{Name:redstone_block},Passengers:[{id:armor_stand,Health:0,Passengers:[{id:falling_block,Time:1,BlockState:{Name:activator_rail},Passengers:[{id:command_block_minecart,Command:'gamerule commandBlockOutput false'},{id:command_block_minecart,Command:'data merge block ~ ~-2 ~ {auto:0}'},{id:command_block_minecart,Command:'fill -801 60 -43 -801 60 -43 diamond_block replace air'},{id:command_block_minecart,Command:'fill -796 60 -43 -796 60 -43 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 60 -38 -791 60 -38 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 60 -36 -790 60 -36 diamond_block replace air'},{id:command_block_minecart,Command:'fill -730 60 33 -730 60 33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -801 61 -26 -801 61 -26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 61 -34 -790 61 -34 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 61 -37 -791 61 -33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 61 -45 -792 61 -33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 61 -45 -793 61 -30 diamond_block replace air'},{id:command_block_minecart,Command:'fill -812 61 -27 -810 61 -27 diamond_block replace air'},{id:command_block_minecart,Command:'fill -817 61 -47 -815 61 -29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -800 61 -43 -797 61 -43 diamond_block replace air'},{id:command_block_minecart,Command:'fill -813 61 -28 -809 61 -28 diamond_block replace air'},{id:command_block_minecart,Command:'fill -795 61 -43 -791 61 -39 diamond_block replace air'},{id:command_block_minecart,Command:'fill -806 61 -43 -802 61 -28 diamond_block replace air'},{id:command_block_minecart,Command:'fill -826 61 -48 -821 61 -48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -814 61 -43 -807 61 -29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -803 61 -42 -794 61 -27 diamond_block replace air'},{id:command_block_minecart,Command:'fill -829 61 -26 -819 61 -26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -810 61 -47 -799 61 -47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 61 -47 -820 61 -28 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 61 -43 -817 61 -27 diamond_block replace air'},{id:command_block_minecart,Command:'fill -819 61 -46 -794 61 -44 diamond_block replace air'},{id:command_block_minecart,Command:'fill -797 61 -17 -797 61 -17 diamond_block replace air'},{id:command_block_minecart,Command:'fill -796 61 63 -794 61 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 61 63 -790 61 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 61 -15 -790 61 -15 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 61 58 -780 61 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -773 61 62 -773 61 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -768 61 61 -768 61 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 61 53 -779 61 55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -782 61 54 -782 61 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -781 61 53 -781 61 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 61 50 -780 61 54 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 61 53 -785 61 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -775 61 55 -775 61 61 diamond_block replace air'},{id:command_block_minecart,Command:'fill -783 61 51 -783 61 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -784 61 49 -784 61 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -777 61 53 -776 61 59 diamond_block replace air'},{id:command_block_minecart,Command:'fill -778 61 51 -777 61 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 61 50 -783 61 50 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 61 50 -777 61 50 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 61 60 -782 61 61 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 61 59 -781 61 59 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 61 62 -781 61 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -774 61 58 -770 61 59 diamond_block replace air'},{id:command_block_minecart,Command:'fill -786 61 63 -781 61 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 61 57 -774 61 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -774 61 60 -769 61 61 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 61 52 -776 61 52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 61 48 -785 61 48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 61 58 -780 61 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 61 48 -778 61 48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -726 61 57 -726 61 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 61 41 -720 61 41 diamond_block replace air'},{id:command_block_minecart,Command:'fill -730 61 34 -730 61 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -731 61 33 -731 61 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -721 61 33 -721 61 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -733 61 34 -732 61 34 diamond_block replace air'},{id:command_block_minecart,Command:'fill -734 61 37 -732 61 53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -735 61 36 -732 61 36 diamond_block replace air'},{id:command_block_minecart,Command:'fill -729 61 33 -722 61 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -733 61 53 -720 61 55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -734 61 35 -720 61 35 diamond_block replace air'},{id:command_block_minecart,Command:'fill -806 62 -50 -806 62 -50 diamond_block replace air'},{id:command_block_minecart,Command:'fill -794 62 -48 -794 62 -47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -819 62 -47 -818 62 -47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -814 62 -47 -811 62 -47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -798 62 -47 -795 62 -47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -826 62 -50 -821 62 -50 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 62 -50 -827 62 -48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -820 62 -48 -797 62 -48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -826 62 -49 -798 62 -49 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 62 63 -793 62 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -789 62 41 -789 62 41 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 62 49 -785 62 49 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 62 51 -785 62 51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -784 62 48 -784 62 48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -782 62 53 -782 62 53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 51 -779 62 51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 56 -779 62 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 58 -779 62 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -777 62 48 -777 62 48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -722 62 32 -722 62 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 38 -720 62 38 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 46 -720 62 46 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 49 -720 62 49 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 52 -720 62 52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 43 -720 62 44 diamond_block replace air'},{id:command_block_minecart,Command:'fill -719 62 57 -719 62 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -811 62 -5 -811 62 -3 diamond_block replace air'},{id:command_block_minecart,Command:'fill -807 62 -28 -807 62 -26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 62 -21 -790 62 -19 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 62 55 -780 62 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 56 -720 62 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -719 62 53 -719 62 55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -794 62 -26 -794 62 -13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -808 62 -28 -808 62 -13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 62 -29 -793 62 -12 diamond_block replace air'},{id:command_block_minecart,Command:'fill -809 62 -27 -809 62 -9 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 62 43 -792 62 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -824 62 -14 -824 62 12 diamond_block replace air'},{id:command_block_minecart,Command:'fill -812 62 -26 -812 62 0 diamond_block replace air'},{id:command_block_minecart,Command:'fill -774 62 29 -774 62 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -769 62 32 -769 62 59 diamond_block replace air'},{id:command_block_minecart,Command:'fill -828 62 32 -828 62 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -768 62 32 -768 62 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -829 62 31 -829 62 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -813 62 -27 -813 62 3 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 62 29 -832 62 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -814 62 -28 -814 62 7 diamond_block replace air'},{id:command_block_minecart,Command:'fill -815 62 -28 -815 62 10 diamond_block replace air'},{id:command_block_minecart,Command:'fill -829 62 -25 -829 62 15 diamond_block replace air'},{id:command_block_minecart,Command:'fill -816 62 -28 -816 62 13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -817 62 -26 -817 62 16 diamond_block replace air'},{id:command_block_minecart,Command:'fill -830 62 -26 -830 62 17 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 62 -26 -831 62 18 diamond_block replace air'},{id:command_block_minecart,Command:'fill -818 62 -26 -818 62 19 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 62 -27 -832 62 20 diamond_block replace air'},{id:command_block_minecart,Command:'fill -803 62 -26 -802 62 -26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -782 62 58 -781 62 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -789 62 37 -788 62 38 diamond_block replace air'},{id:command_block_minecart,Command:'fill -796 62 -26 -795 62 -14 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 62 -24 -791 62 -11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -811 62 -26 -810 62 -6 diamond_block replace air'},{id:command_block_minecart,Command:'fill -782 62 30 -781 62 51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 62 30 -830 62 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 62 59 -778 62 59 diamond_block replace air'},{id:command_block_minecart,Command:'fill -806 62 -27 -804 62 -26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 31 -777 62 33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -804 62 42 -802 62 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -807 62 42 -805 62 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -789 62 43 -787 62 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -788 62 40 -786 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -764 62 36 -762 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -767 62 33 -765 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -772 62 62 -769 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -725 62 57 -722 62 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -786 62 30 -783 62 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -800 62 -26 -797 62 -18 diamond_block replace air'},{id:command_block_minecart,Command:'fill -783 62 31 -780 62 48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -775 62 30 -772 62 54 diamond_block replace air'},{id:command_block_minecart,Command:'fill -828 62 -25 -825 62 13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -825 62 27 -821 62 29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -823 62 -14 -819 62 13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -739 62 34 -734 62 34 diamond_block replace air'},{id:command_block_minecart,Command:'fill -737 62 33 -732 62 33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -781 62 60 -776 62 61 diamond_block replace air'},{id:command_block_minecart,Command:'fill -825 62 23 -820 62 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -824 62 14 -819 62 22 diamond_block replace air'},{id:command_block_minecart,Command:'fill -801 62 42 -796 62 61 diamond_block replace air'},{id:command_block_minecart,Command:'fill -783 62 49 -777 62 49 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 62 62 -774 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -741 62 35 -735 62 35 diamond_block replace air'},{id:command_block_minecart,Command:'fill -730 62 32 -724 62 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -776 62 31 -770 62 51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -820 62 29 -812 62 29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -799 62 44 -790 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -748 62 36 -736 62 36 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 34 -764 62 34 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 35 -763 62 35 diamond_block replace air'},{id:command_block_minecart,Command:'fill -771 62 48 -755 62 53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -778 62 37 -761 62 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -754 62 37 -735 62 53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -827 62 42 -808 62 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -824 62 -25 -798 62 -17 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 62 61 -804 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -824 62 -16 -797 62 -15 diamond_block replace air'},{id:command_block_minecart,Command:'fill -785 62 38 -755 62 47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -827 62 41 -795 62 41 diamond_block replace air'},{id:command_block_minecart,Command:'fill -827 62 40 -794 62 40 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 62 63 -797 62 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -827 62 38 -792 62 39 diamond_block replace air'},{id:command_block_minecart,Command:'fill -827 62 33 -790 62 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -773 62 54 -734 62 55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -826 62 30 -787 62 36 diamond_block replace air'},{id:command_block_minecart,Command:'fill -761 62 58 -721 62 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -773 62 56 -732 62 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -761 62 59 -718 62 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -773 62 57 -727 62 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 62 63 -718 62 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 62 -46 -792 62 -46 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 62 -32 -791 62 -32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 62 -44 -791 62 -44 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 62 -33 -790 62 -33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -782 62 53 -782 62 53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -782 62 58 -781 62 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 56 -779 62 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 62 55 -780 62 57 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 62 51 -779 62 51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -730 62 32 -724 62 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -722 62 32 -722 62 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 38 -720 62 38 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 43 -720 62 44 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 46 -720 62 46 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 49 -720 62 49 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 62 52 -720 62 52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -719 62 53 -719 62 55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 63 -47 -792 63 -47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 63 -53 -832 63 -51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 63 -49 -793 63 -47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 63 -47 -791 63 -45 diamond_block replace air'},{id:command_block_minecart,Command:'fill -796 63 -48 -795 63 -48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -830 63 -53 -827 63 -53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -797 63 -49 -794 63 -49 diamond_block replace air'},{id:command_block_minecart,Command:'fill -805 63 -50 -796 63 -50 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 63 -52 -820 63 -52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -820 63 -50 -807 63 -50 diamond_block replace air'},{id:command_block_minecart,Command:'fill -817 63 -52 -799 63 -52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 63 -51 -798 63 -51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 63 21 -832 63 28 diamond_block replace air'},{id:command_block_minecart,Command:'fill -829 63 16 -829 63 30 diamond_block replace air'},{id:command_block_minecart,Command:'fill -828 63 14 -828 63 31 diamond_block replace air'},{id:command_block_minecart,Command:'fill -827 63 14 -827 63 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -826 63 14 -825 63 17 diamond_block replace air'},{id:command_block_minecart,Command:'fill -830 63 18 -825 63 22 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 63 19 -826 63 29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -824 63 13 -824 63 13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -794 63 -12 -794 63 -12 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 63 -11 -793 63 -11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 63 -10 -792 63 -10 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -13 -790 63 -13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -11 -790 63 -11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -787 63 37 -787 63 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 63 -10 -791 63 -9 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 63 36 -779 63 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -816 63 14 -816 63 16 diamond_block replace air'},{id:command_block_minecart,Command:'fill -805 63 -10 -805 63 -8 diamond_block replace air'},{id:command_block_minecart,Command:'fill -774 63 23 -774 63 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -776 63 25 -776 63 30 diamond_block replace air'},{id:command_block_minecart,Command:'fill -775 63 24 -775 63 29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -806 63 -14 -806 63 -5 diamond_block replace air'},{id:command_block_minecart,Command:'fill -814 63 8 -814 63 20 diamond_block replace air'},{id:command_block_minecart,Command:'fill -807 63 -14 -807 63 -2 diamond_block replace air'},{id:command_block_minecart,Command:'fill -813 63 4 -813 63 17 diamond_block replace air'},{id:command_block_minecart,Command:'fill -812 63 1 -812 63 14 diamond_block replace air'},{id:command_block_minecart,Command:'fill -811 63 -2 -811 63 11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -810 63 -5 -810 63 8 diamond_block replace air'},{id:command_block_minecart,Command:'fill -809 63 -8 -809 63 5 diamond_block replace air'},{id:command_block_minecart,Command:'fill -808 63 -12 -808 63 1 diamond_block replace air'},{id:command_block_minecart,Command:'fill -815 63 11 -815 63 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -778 63 36 -777 63 36 diamond_block replace air'},{id:command_block_minecart,Command:'fill -805 63 -12 -804 63 -11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -771 63 25 -770 63 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -817 63 17 -816 63 19 diamond_block replace air'},{id:command_block_minecart,Command:'fill -773 63 24 -772 63 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -769 63 26 -767 63 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -818 63 20 -816 63 22 diamond_block replace air'},{id:command_block_minecart,Command:'fill -819 63 23 -816 63 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -780 63 26 -777 63 30 diamond_block replace air'},{id:command_block_minecart,Command:'fill -764 63 33 -759 63 33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -760 63 37 -755 63 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -763 63 34 -757 63 34 diamond_block replace air'},{id:command_block_minecart,Command:'fill -762 63 35 -756 63 35 diamond_block replace air'},{id:command_block_minecart,Command:'fill -767 63 32 -760 63 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -805 63 -14 -797 63 -14 diamond_block replace air'},{id:command_block_minecart,Command:'fill -769 63 31 -761 63 31 diamond_block replace air'},{id:command_block_minecart,Command:'fill -771 63 30 -762 63 30 diamond_block replace air'},{id:command_block_minecart,Command:'fill -805 63 -13 -795 63 -13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -773 63 29 -763 63 29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 63 25 -781 63 25 diamond_block replace air'},{id:command_block_minecart,Command:'fill -761 63 36 -749 63 36 diamond_block replace air'},{id:command_block_minecart,Command:'fill -814 63 24 -793 63 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -811 63 26 -781 63 29 diamond_block replace air'},{id:command_block_minecart,Command:'fill -820 63 27 -766 63 27 diamond_block replace air'},{id:command_block_minecart,Command:'fill -820 63 28 -765 63 28 diamond_block replace air'},{id:command_block_minecart,Command:'fill -804 63 58 -804 63 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -803 63 58 -802 63 61 diamond_block replace air'},{id:command_block_minecart,Command:'fill -807 63 59 -805 63 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -803 63 62 -800 63 62 diamond_block replace air'},{id:command_block_minecart,Command:'fill -786 63 38 -786 63 38 diamond_block replace air'},{id:command_block_minecart,Command:'fill -787 63 37 -787 63 38 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 63 38 -790 63 43 diamond_block replace air'},{id:command_block_minecart,Command:'fill -795 63 43 -793 63 43 diamond_block replace air'},{id:command_block_minecart,Command:'fill -794 63 41 -792 63 41 diamond_block replace air'},{id:command_block_minecart,Command:'fill -789 63 39 -786 63 39 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 63 40 -789 63 40 diamond_block replace air'},{id:command_block_minecart,Command:'fill -795 63 42 -789 63 42 diamond_block replace air'},{id:command_block_minecart,Command:'fill -794 63 -12 -794 63 -12 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 63 -11 -793 63 -11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 63 -31 -792 63 -25 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 63 -10 -792 63 -10 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 63 -10 -791 63 -9 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -40 -790 63 -40 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -37 -790 63 -37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -35 -790 63 -35 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -32 -790 63 -32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -23 -790 63 -22 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -17 -790 63 -16 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -13 -790 63 -13 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 63 -11 -790 63 -11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -779 63 36 -779 63 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -778 63 36 -777 63 36 diamond_block replace air'},{id:command_block_minecart,Command:'fill -748 63 35 -742 63 35 diamond_block replace air'},{id:command_block_minecart,Command:'fill -741 63 34 -740 63 34 diamond_block replace air'},{id:command_block_minecart,Command:'fill -738 63 33 -738 63 33 diamond_block replace air'},{id:command_block_minecart,Command:'fill -736 63 32 -736 63 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -733 63 32 -731 63 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -723 63 32 -723 63 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -721 63 32 -721 63 32 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 63 36 -720 63 37 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 63 39 -720 63 40 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 63 45 -720 63 45 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 63 47 -720 63 47 diamond_block replace air'},{id:command_block_minecart,Command:'fill -720 63 51 -720 63 51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -719 63 52 -719 63 52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -719 63 56 -719 63 56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -718 63 56 -718 63 58 diamond_block replace air'},{id:command_block_minecart,Command:'fill -717 63 57 -716 63 60 diamond_block replace air'},{id:command_block_minecart,Command:'fill -717 63 61 -715 63 63 diamond_block replace air'},{id:command_block_minecart,Command:'fill -718 63 53 -718 63 54 diamond_block replace air'},{id:command_block_minecart,Command:'fill -831 64 -53 -831 64 -53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -815 64 -55 -815 64 -55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 64 -50 -792 64 -48 diamond_block replace air'},{id:command_block_minecart,Command:'fill -819 64 -52 -818 64 -52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -830 64 -56 -828 64 -56 diamond_block replace air'},{id:command_block_minecart,Command:'fill -823 64 -55 -821 64 -55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -798 64 -52 -796 64 -52 diamond_block replace air'},{id:command_block_minecart,Command:'fill -797 64 -51 -795 64 -51 diamond_block replace air'},{id:command_block_minecart,Command:'fill -795 64 -50 -793 64 -50 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 64 -55 -825 64 -55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -812 64 -55 -804 64 -55 diamond_block replace air'},{id:command_block_minecart,Command:'fill -826 64 -53 -797 64 -53 diamond_block replace air'},{id:command_block_minecart,Command:'fill -832 64 -54 -798 64 -54 diamond_block replace air'},{id:command_block_minecart,Command:'fill -793 64 -10 -793 64 -10 diamond_block replace air'},{id:command_block_minecart,Command:'fill -792 64 -9 -792 64 -9 diamond_block replace air'},{id:command_block_minecart,Command:'fill -791 64 -8 -791 64 -8 diamond_block replace air'},{id:command_block_minecart,Command:'fill -790 64 -12 -790 64 -12 diamond_block replace air'},{id:command_block_minecart,Command:'fill -810 64 9 -810 64 11 diamond_block replace air'},{id:command_block_minecart,Command:'fill -801 64 -10 -801 64 -7 diamond_block replace air'},{id:command_block_minecart,Command:'fill -767 64 20 -767 64 23 diamond_block replace air'},{id:command_block_minecart,Command:'fill -777 64 20 -777 64 25 diamond_block replace air'},{id:command_block_minecart,Command:'fill -776 64 19 -776 64 24 diamond_block replace air'},{id:command_block_minecart,Command:'fill -775 64 18 -775 64 23 diamond_block replace air'},{id:command_block_minecart,Command:'fill -774 64 17 -774 64 22 diamond_block replace air'},{id:command_block_minecart,Command:'fill -766 64 21 -766 64 26 diamond_block replace air'},{id:command_block_minecart,Command:'fill -765 64 22 -765 64 27 diamond_block replace air'},{id:command_block_minecart,Command:'fill -764 64 23 -764 64 28 diamond_block replace air'},{id:command_block_minecart,Command:'fill -802 64 -12 -802 64 -4 diamond_block replace air'},{id:command_block_minecart,Command:'fill -809 64 6 -809 64 17 diamond_block replace air'},{id:command_block_minecart,Command:'fill -803 64 -12 -803 64 -1 diamond_block replace air'},{id:command_block_minecart,Command:'fill -808 64 2 -808 64 15 diamond_block replace air'},{id:command_block_minecart,Command:'fill -807 64 -1 -807 64 12 diamond_block replace air'},{id:command_block_minecart,Command:'fill -806 64 -4 -806 64 9 diamond_block replace air'},{id:command_block_minecart,Command:'fill -805 64 -7 -805 64 6 diamond_block replace air'},{id:command_block_minecart,Command:'fill -804 64 -10 -804 64 3 diamond_block replace air'},{id:command_block_minecart,Command:'fill -811 64 12 -810 64 14 diamond_block replace air'},{id:command_block_minecart,Command:'fill -773 64 18 -772 64 23 diamond_block replace air'},{id:command_block_minecart,Command:'setblock ~ ~1 ~ command_block{auto:1,Command:"fill ~ ~ ~ ~ ~-2 ~ air"}'},{id:command_block_minecart,Command:'kill @e[type=command_block_minecart,distance=..1]'}]}]}]}